In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from llm import generate_responses, generate_prompt

def render_comparison(data):
    html = """
    <style>
        table { border-collapse: collapse; width: 100%; font-family: sans-serif; }
        th, td { border: 1px solid #ccc; padding: 8px; text-align: left; vertical-align: top; width: 25%; }
        th { background-color: #f2f2f2; }
        td pre { white-space: pre-wrap; word-break: break-word; }
    </style>
    <table>
        <tr><th>Problem ID</th><th>Original Plan</th><th>NL Description</th><th>New Plan</th></tr>
    """
    for item in data:
        line = "<tr>"
        for i, cell in enumerate(item):
            if i == 0:
                line += f"<td>{cell}</td>"
            else:
                line += f"<td><pre>{cell}</pre></td>"
        line += "</tr>"
        html += line
    html += "</table>"
    return html


In [2]:
plan_file_path = "../../data/plans/plans.txt"
with open(plan_file_path, 'r') as f:
    with open("transport_p2nl.txt", 'w') as f2:
        lines = f.readlines()
        count = 0
        for line in lines:
            if count >20:
                break
            item = line.split("&&")
            domain = item[0].strip()
            prob = item[1].strip()
            if domain != "transport":
                continue
            plan = line.split("&&")[-1].strip()
            print("Processing:", prob)
            prompt_names = ['p2nl', 'p2nl-summary', 'p2nl-detail']
            for prompt_name in prompt_names:
                prompt = generate_prompt(prompt_name, {'plan': plan})
                res = generate_responses('gpt-4o-mini', prompt)
                f2.write(f"{prob}&&{plan}&&{prompt}&&{res['content']}\n")
                f2.flush()
            count += 1


Processing: p01.pddl


RuntimeError: OpenAI API call failed: Connection error.

In [ ]:
data = []
with open("transport_p2nl.txt", 'r') as f:
        lines = f.readlines()
        for line in lines:
            item = line.split("&&")
            data.append(item)


In [5]:
new_data = []
with open("transport_nl2plan.txt", 'w') as f:
    for item in data:
        print(f"Processing Problem ID: {item[0]}")
        prob = item[0].strip()
        plan = item[1].strip()
        nl = item[2].strip()
        res = generate_responses('gpt-4o-mini', 'nl2plan', prompt_args={
            'nl': nl
        })
        f.write(f"{prob}&&{plan}&&{nl}&&{res['content']}\n")
        f.flush()
        new_data.append([prob, plan, nl, res['content']])


    
    

Processing Problem ID: p01.pddl
Processing Problem ID: p02.pddl
Processing Problem ID: p03.pddl
Processing Problem ID: p04.pddl
Processing Problem ID: p05.pddl
Processing Problem ID: p06.pddl
Processing Problem ID: p07.pddl
Processing Problem ID: p08.pddl
Processing Problem ID: p09.pddl
Processing Problem ID: p10.pddl
Processing Problem ID: p11.pddl
Processing Problem ID: p12.pddl
Processing Problem ID: p13.pddl
Processing Problem ID: p14.pddl
Processing Problem ID: p15.pddl
Processing Problem ID: p16.pddl
Processing Problem ID: p17.pddl
Processing Problem ID: p18.pddl
Processing Problem ID: p19.pddl
Processing Problem ID: p20.pddl
Processing Problem ID: p21.pddl


In [6]:

html = render_comparison(new_data)
with open("nl2plan_output.html", "w", encoding="utf-8") as f:
    f.write(html)